In [ ]:
! pip install chromadb langchain-openai
! pip install langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 22.2 MB/s eta 0:00:00

  Attempting uninstall: requests

    Found existing installation: requests 2.32.4

    Uninstalling requests-2.32.4:

      Successfully uninstalled requests-2.32.4

   ---------------------------------------- 0/7 [requests]
   ---------------------------------------- 0/7 [requests]
   ----------- ---------------------------- 2/7 [pydantic-settings]
   ----------- ---------------------------- 2/7 [pydantic-settings]
  Attempting uninstall: langchain-core
   ----------- ---------------------------- 2/7 [pydantic-settings]
   ----------------- ---------------------- 3/7 [langchain-core]
    Found existing installation: langchain-core 0.3.78
   ----------------- ---------------------- 3/7 [langchain-

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.26 requires langchain-core<1.0.0,>=0.3.66, but you have langchain-core 1.0.4 which is incompatible.
langchain 0.3.26 requires langchain-text-splitters<1.0.0,>=0.3.8, but you have langchain-text-splitters 1.0.0 which is incompatible.
langchain-ollama 0.3.4 requires langchain-core<1.0.0,>=0.3.68, but you have langchain-core 1.0.4 which is incompatible.
llama-index-core 0.13.4 requires setuptools>=80.9.0, but you have setuptools 70.2.0 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv("../../config/local.env")

c:\Users\ytiam\notebooks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ytiam\notebooks\.venv\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find cuobjdump.exe
  warnings.warn(f"Failed to find {binary}")
c:\Users\ytiam\notebooks\.venv\Lib\site-packages\triton\knobs.py:212: UserWarning: Failed to find nvdisasm.exe
  warnings.warn(f"Failed to find {binary}")


True

In [2]:
emb = OpenAIEmbeddings(model="text-embedding-3-small")
memory = Chroma(collection_name="agent_memory", embedding_function=emb)

C:\Users\ytiam\AppData\Local\Temp\ipykernel_28888\2335659448.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  memory = Chroma(collection_name="agent_memory", embedding_function=emb)


In [3]:
from langchain.tools import tool
import wikipedia

@tool
def wiki_search(query: str) -> str:
    """Search Wikipedia and return a short summary."""
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception:
        return "No summary found."

tools = [wiki_search]


In [4]:
from langchain_core.messages import SystemMessage

PLANNER_PROMPT = SystemMessage(
    content="Plan research steps. Do not perform research."
)

RESEARCHER_PROMPT = SystemMessage(
    content=(
        "You are the Research Agent. If needed, call tools. "
        "Also, retrieve memory and use it. "
        "If memory helps answer directly, use it instead of calling tools."
    )
)

WRITER_PROMPT = SystemMessage(
    content="Write a final coherent answer based on previous messages."
)


In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import ToolMessage

llm = ChatOpenAI(model="gpt-4o-mini")

def researcher_node(state):
    messages = [RESEARCHER_PROMPT] + state["messages"]

    # Step 1: Retrieve memory
    last_user_query = state["messages"][0].content
    recalled = memory.similarity_search(last_user_query, k=2)
    memory_context = "\n".join([doc.page_content for doc in recalled])

    # Step 2: Provide memory context to model
    messages.append({"role": "system", "content": f"Relevant memory:\n{memory_context}"})

    # Step 3: Let LLM decide whether to call tool
    response = llm.bind_tools(tools).invoke(messages)

    # Step 4: If tool call → execute and append ToolMessage
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        result = wiki_search.invoke(tool_call["args"])
        tool_msg = ToolMessage(content=result, tool_call_id=tool_call["id"])
        new_messages = state["messages"] + [response, tool_msg]
    else:
        new_messages = state["messages"] + [response]

    # Step 5: Save message to memory
    memory.add_texts([new_messages[-1].content])

    return {"messages": new_messages}


In [9]:
from langgraph.graph import StateGraph, MessagesState, END

def planner_node(state):
    response = llm.invoke([PLANNER_PROMPT, state["messages"][-1]])
    return {"messages": state["messages"] + [response]}

def writer_node(state):
    response = llm.invoke([WRITER_PROMPT] + state["messages"])
    return {"messages": state["messages"] + [response]}

builder = StateGraph(MessagesState)
builder.set_entry_point("planner")
builder.add_node("planner", planner_node)
builder.add_node("researcher", researcher_node)
builder.add_node("writer", writer_node)

builder.add_edge("planner", "researcher")
builder.add_edge("researcher", "writer")
builder.add_edge("writer", END)

graph = builder.compile()

In [10]:
query = "Explain how solar panels work again."
result = graph.invoke({"messages": [{"role": "user", "content": query}]})

print("\n🤖 Final Answer:\n", result["messages"][-1].content)



🤖 Final Answer:
 Sure! Here’s a detailed explanation of how solar panels work:

### 1. Solar Cells:
Solar panels are composed of many solar cells, primarily made from silicon. These cells are designed to capture sunlight efficiently.

### 2. Photovoltaic Effect:
The key mechanism behind solar panels is the photovoltaic effect. When sunlight, which consists of energy particles called photons, strikes the solar cells, it energizes the electrons within the silicon. This energy causes the electrons to break free from their atoms, creating electrical current.

### 3. Electricity Generation:
- As electrons move, they generate a flow of electric current (direct current or DC).
- This DC electricity is then channeled through wiring in the solar panels.

### 4. Inverter:
- The generated DC electricity needs to be converted to alternating current (AC), which is the standard form of electricity used in homes and for most appliances. This conversion is handled by an inverter.

### 5. Energy Stora